# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Xinyao Niu
###### Python version: 3.6.8
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [1]:
# version of python
!python3 --version

Python 3.6.8 :: Anaconda, Inc.


In [2]:
import numpy as np
import os
from os.path import join
import subprocess
import pandas as pd
from collections import defaultdict as dd
from math import log

In [3]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        if (line.split('.')[-1] != "txt") and (line != "test.csv"):
            ls.append(line)
    return ls

In [4]:
datapath = join(os.getcwd(),'2019S1-proj1-data')
files = bash_call(['ls', datapath])
print(files)

['anneal.csv', 'breast-cancer.csv', 'car.csv', 'cmc.csv', 'hepatitis.csv', 'hypothyroid.csv', 'mushroom.csv', 'nursery.csv', 'primary-tumor.csv']


In [5]:
datapath

'/Users/xinyaoniu/Documents/COMP30027-ML/prject 1/2019S1-proj1-data'

In [6]:
HEADERS = {
        "anneal.csv": "family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,surface-finish,surface-quality,enamelability,bc,bf,bt,bw-me,bl,m,chrom,phos,cbond,marvi,exptl,ferro,corr,bbvc,lustre,jurofm,s,p,shape,oil,bore,packing,class".split(","),
        "breast-cancer.csv": "age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class".split(","),
        "car.csv": "buying,maint,doors,persons,lug_boot,safety,class".split(","),
        "cmc.csv": "w-education,h-education,n-child,w-relation,w-work,h-occupation,standard-of-living,media-exposure,class".split(","),
        "hepatitis.csv": "sex,steroid,antivirals,fatigue,malaise,anorexia,liver-big,liver-firm,spleen-palpable,spiders,ascites,varices,histology,class".split(","),
        "hypothyroid.csv": "sex,on-thyroxine,query-on-thyroxine,on_antithyroid,surgery,query-hypothyroid,query-hyperthyroid,pregnant,sick,tumor,lithium,goitre,TSH,T3,TT4,T4U,FTI,TBG,class".split(","),
        "mushroom.csv": "cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class".split(","),
        "nursery.csv": "parents,has_nurs,form,children,housing,finance,social,health,class".split(","),
        "primary-tumor.csv": "age,sex,histologic-type,degree-of-diffe,bone,bone-marrow,lung,pleura,peritoneum,liver,brain,skin,neck,supraclavicular,axillar,mediastinum,abdominal,class".split(","),
        "test.csv":["0","1","2","3","4"],
    }

## Assumption that I made for this project (details about how to process refer to code comments)
- For better performance, missing values has been processed. 
- Using epsilon smoothing as a default smoothing method, also provide entrence for laplace smoothing.
- all of the acc calculated are based on above assumptions
- When calculating IG, missing values were not counted.

In [7]:
class naive_bayes_learner():
    
    def __init__(self, path:str, file:str):
        self.file = file
        self.path = join(path, file)
        self.df = None  # primary dataframe
        self.labels = []  # possible labels for the class
        self.attributes = []  # possible attributes for each row
        self.last = -1  # index of the labels
        
        self.prob = []  # the model hyper-parameters
        self.missing = []  # number of missing value in each attributes
        self.label_prob = []  # the probability for the prior
        self.test_size = -1
        self.train_size = -1
        self.train_set = pd.DataFrame()
        self.test_set = pd.DataFrame()
        
        self.predictions = []  # contains the prediction results in a form of (prediction, real labels)
        
        self.acc = 0  # accuracy of the predictions
        
        self.ig = {}
        
        
    def preprocess(self, val:str='hold-out', shuffle:bool=False, hold:float=0.2) -> None:
        """
        open a csv and transform it into a usable format
        
        *val -- define the evaluation method, it use hold out in defualt
        
        *shuffle -- whether we want to shuffle the dataset, it set to false in defaut

        *hold -- if the val method is hold-out, this parameter can change the proportion between
        training set and test set, it set to 0.2 in default
        """
        self.df = pd.read_csv(self.path, header=None)
        self.df.columns = HEADERS[self.file]
        self.size = len(self.df.index)
        # labels location
        self.last = self.df.columns[-1]
        # remove duplicate labels
        self.labels = list(set(self.df[self.last]))
        
        if shuffle or val=="random sampling":
            self.df = self.df.sample(frac=1).reset_index(drop=True)
        
        if((val=="random sampling") and (hold != 0)):
            self.test_size = int(self.df.shape[0] * hold)
            self.train_size = self.df.shape[0] - self.test_size

            self.train_set = self.df.head(self.train_size)
            self.test_set = self.df.tail(self.test_size)
        elif ((val=="hold-out") and (hold != 0)):
            self.train_set = pd.DataFrame.copy(self.df)
            for l in self.labels:
                # randomly sampling out test set from the real data set
                select = self.train_set[self.train_set[self.last] == l].sample(frac=hold)
                # drop the testing set
                self.train_set = self.train_set.drop(select.index, axis=0)
                self.test_set = self.test_set.append(select)

            # reset the indexes
            self.train_set = self.train_set.reset_index(drop=True)
            self.test_set = self.test_set.reset_index(drop=True)
            
            self.test_size = self.df.shape[0]
            self.train_size = self.df.shape[0]
        else:
            self.train_set = self.df
            self.test_set = self.df
            self.test_size = self.df.shape[0]
            self.train_size = self.df.shape[0]
        
        #print("train=", self.train_set.shape, "test=", self.test_set.shape)

        # pre-process the train set
        for i in self.train_set.columns:
            stats = self.count_frequency(self.train_set[[i,self.last]])
            self.attributes.append(tuple(set(self.train_set[i])))
            self.prob.append(stats)
            #print("len of prob =", len(self.prob))
        #print("len of header =", len(self.train_set.columns))
            
    
    def count_frequency(self, df:pd.DataFrame ,accumulate:dict={}, laplace:bool=False) -> dd:
        """
        helper function for preprocess
        counting the frequency for each different value in an array
        
        `df` -- input dataframe that need to be processed, normally contains two columns
        and one of it is labels
        
        `accumulate` -- the count number that might add on the top of the current on, it
        is empty by default
        
        `laplace` -- whether we are going to use laplace smoothing or not
        """
        
        if(laplace):
            res = dd(lambda:1, accumulate)
        else:
            res = dd(lambda:0,accumulate)
            
        miss = dd(lambda:0)
        
        index = df.columns
        same = False
        
        # rename the index if they are the same
        if index[0] == index[1]:
            index = ['one', 'two']
            df.columns = index
            same = True
            
        for i in range(df.shape[0]):
            feature = df[index[0]].loc[i]
            label = df[index[1]].loc[i]
            if same:
                res[str(label)] += 1
            elif str(feature) == '?': 
                #ignore the missing value
                miss[str(label)] += 1
            else:
                res[str(feature)+'|'+ str(label)] += 1
        
        self.missing.append(miss)
        
        return res
    
    
    def find_condition(self, prob:str) -> str:
        """
        Helper function of train
        finding the condition of a conditional probability
        
        `prob` -- the conditional probability
        """
        return prob.split("|")[1]
    
    
    def train(self) -> None:
        """
        Calculating the conditional probability for NB
        """
        label = self.prob[-1]
        #print(label)
        for i in range(len(self.prob)-1):
            current = self.prob[i]
            missing = self.missing[i]
            #print(current)
            for k,v in current.items():
                
                real = label[self.find_condition(k)] - missing[self.find_condition(k)]
                
                #if(k=="2|A"):
                    #print("2|A", current[k],label[self.find_condition(k)], missing[self.find_condition(k)], real)
                    
                if(real != 0):
                    current[k] /= real
                else:
                    current[k] = 0
                    
        for k,v in label.items():
            label[k] /= self.train_size
        
        self.prob[-1] = label
    
    
    def predict(self) -> None:
        """
        predicting the class for an instance or a set of instances, basd on a trained model
        """
        
        features = self.test_set[self.test_set.columns[:-1]]
        results = self.test_set[self.test_set.columns[-1]]
        
        for i in features.index:
            p = np.zeros(len(self.labels))
            for l in range(len(self.labels)):
                p[l] = (self.prob_calculator(features.loc[i], self.labels[l]))
            #print(features.loc[i])
            #print(self.labels)
            #print(p, self.labels[np.argmax(p)], results.loc[i])
            self.predictions.append((self.labels[np.argmax(p)], results.loc[i]))
    
    
    def prob_calculator(self, features:pd.DataFrame, target:str) -> float:
        """
        helper function of predict
        calculate the probability based on the given features
        
        `features` -- dataframe that needs to calculate the probability
        
        `target` -- given label
        """
        
        # epsilon for epsilon smoothing
        eps = 0.0001
        
        if(self.prob[-1][target] != 0):
            res = log(self.prob[-1][target],2)
        else:
            res = log(eps,2)
        n = 0
        for f in features:
            if str(f) != '?':
                condition = str(f) + '|' + target
                # using epsilon smoothing in default
                if (self.prob[n][condition] <= 0):
                    res += log(eps,2)
                else:
                    res += log(self.prob[n][condition],2)
            n += 1
        return res


    def evaluate(self, acc_v:bool) -> None:
        """
        evaluate a set of predictions, in a supervised context
        
        `acc_v` -- make it True if you want to see the acc output
        """
        #print(self.predictions)
        total = len(self.predictions)
        correct = 0
        
        for pair in self.predictions:
            if pair[0] == pair[1]:
                correct += 1
                
        self.acc = correct/total
        
        if acc_v:
            print("correct =", correct, "total =",total)
            print("acc =",correct/total)
        
    
    def info_gain(self) -> None:
        """
        calculate the information gain of an attributes with respect to the labels
        """
        full = []  # contains the frequency of each atttribute along their column
        for i in self.test_set:
            full.append(self.count_frequency(self.df[[i,self.last]]))
        
        parent = 0
        n = self.df.shape[0]
        
        for i in full[-1].values():
            parent += self.entropy(i/n)
        #print("parent=",parent,"n=", n)
        #print(full[-1])
        
        for i in range(len(full)-1):
            current = full[i]
            child = 0
            for attr in self.attributes[i]:
                a = []
                for l in self.labels:
                    # ignore the missing value
                    if attr != "?":
                        target = str(attr) + "|" + l
                        a.append(current[target])
                # weighted entropy for this child node
                child += (sum(a)/self.size)*sum([self.entropy(i/sum(a)) for i in a])
            self.ig[self.df.columns[i]] = parent-child
        
        
    def entropy(self, p:float) -> float:
        """
        helper function for info_gain
        mainly for calculate the entropy for given probability
        
        *p -- given probability
        """
        if(p==0):
            return 0
        return -p*log(p,2)
        
        
    
    def run(self, v="hold-out", h=0.2, s=False, l=False, ig=False, mi=False, acc_v=False) -> None:
        """
        this function will automatically run through the workflow with different
        set up parameters. It is only for convenient testing and debuging purpose.
        """
        
        self.preprocess(shuffle=s, val=v, hold=h)
        
        self.train()
        
        self.predict()
        self.evaluate(acc_v)
        
        if mi:
            print("missing value for each column", [sum(i.values()) for i in self.missing])
            print("number of possible labels =",len(self.labels))
        
        if ig:
            self.info_gain()
            print("\nIG *")
            print(self.ig)
        

In [8]:
for i in files:
    print("now processing file:",i,"****************")
    #test = naive_bayes_learner(datapath, i)
    #test.run(v='random sampling')
    #print("---------------------------------------------")
    test = naive_bayes_learner(datapath, i)
    test.run(v='hold-out')
    print("---------------------------------------------")
    test = naive_bayes_learner(datapath, i)
    test.run(v='full', ig=True, mi=True)
    print("=============================================")

now processing file: anneal.csv ****************
---------------------------------------------
missing value for each column [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
number of possible labels = 5

IG *
{'family': 0.40908953764451006, 'product-type': 0.0, 'steel': 0.3060515354289406, 'carbon': 0.051344088764404106, 'hardness': 0.29108220585994704, 'temper_rolling': 0.14711886228095605, 'condition': 0.2137228803159088, 'formability': 0.29223544065798446, 'strength': 0.1261663361036096, 'non-ageing': 0.14107379163812883, 'surface-finish': 0.032488406491841815, 'surface-quality': 0.43517783626288564, 'enamelability': 0.03870173274881061, 'bc': 0.0004376065202120749, 'bf': 0.03935557414283686, 'bt': 0.021775078259213876, 'bw-me': 0.037997478813511565, 'bl': 0.036703081364408474, 'm': 0.0, 'chrom': 0.11722522630372034, 'phos': 0.029753745208638938, 'cbond': 0.02704235332867677, 'marvi': 0.0, 'exptl': 0.015604780443500665, 'f

In [11]:
summary = pd.DataFrame()

for f in files:
    print("now processing file:",f,"****************")
    h1 = []
    h3 = []
    h5 = []
    r = 0
    for i in range(10):
        test = naive_bayes_learner(datapath, f)
        test.run(v='hold-out',h=0.1)
        h1.append(test.acc)
        test = naive_bayes_learner(datapath, f)
        test.run(v='hold-out',h=0.3)
        h3.append(test.acc)
        test = naive_bayes_learner(datapath, f)
        test.run(v='hold-out',h=0.5)
        h5.append(test.acc)
        
    print([round(j,2) for j in h1])
    print([round(j,2) for j in h3])
    print([round(j,2) for j in h5])
    test = naive_bayes_learner(datapath, f)
    test.run(v='random sampling')
    r = test.acc
    test = naive_bayes_learner(datapath, f)
    test.run(v='full', mi=True)
    summary[f] = pd.Series([r,(round(min(h1),2), round(max(h1),2)), (round(min(h3),2), round(max(h3),2)),(round(min(h5),2), round(max(h5),2)),test.acc, len(test.labels), test.df.shape[0]])


now processing file: anneal.csv ****************
[0.97, 1.0, 0.98, 0.99, 0.99, 0.98, 1.0, 0.99, 0.98, 0.98]
[0.98, 0.99, 0.98, 0.97, 0.99, 0.99, 0.99, 0.98, 0.97, 0.99]
[0.98, 0.98, 0.98, 0.98, 0.99, 0.97, 0.99, 0.99, 0.99, 0.98]
missing value for each column [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
number of possible labels = 5
now processing file: breast-cancer.csv ****************
[0.75, 0.75, 0.82, 0.68, 0.64, 0.57, 0.68, 0.82, 0.75, 0.71]
[0.71, 0.77, 0.72, 0.7, 0.72, 0.78, 0.73, 0.67, 0.69, 0.72]
[0.7, 0.73, 0.71, 0.67, 0.73, 0.7, 0.73, 0.74, 0.74, 0.7]
missing value for each column [0, 0, 0, 0, 8, 0, 0, 1, 0, 0]
number of possible labels = 2
now processing file: car.csv ****************
[0.83, 0.87, 0.86, 0.87, 0.84, 0.86, 0.84, 0.83, 0.83, 0.86]
[0.87, 0.86, 0.84, 0.86, 0.85, 0.84, 0.84, 0.85, 0.86, 0.85]
[0.87, 0.85, 0.86, 0.86, 0.85, 0.85, 0.86, 0.84, 0.85, 0.83]
missing value for each column [0, 0, 0, 0, 0, 

In [12]:
summary.index = ['Random Sampling', '9-1 Hold-out', '7-3 Hold-out', '5-5 Hold-out','Full', '# of labels', '# of instance']

In [13]:
summary

,anneal.csv,breast-cancer.csv,car.csv,cmc.csv,hepatitis.csv,hypothyroid.csv,mushroom.csv,nursery.csv,primary-tumor.csv
Random Sampling,0.96648,0.666667,0.869565,0.445578,0.774194,0.954114,0.971059,0.907793,0.58209
9-1 Hold-out,"(0.97, 1.0)","(0.57, 0.82)","(0.83, 0.87)","(0.44, 0.54)","(0.6, 1.0)","(0.95, 0.95)","(0.96, 0.98)","(0.89, 0.91)","(0.42, 0.64)"
7-3 Hold-out,"(0.97, 0.99)","(0.67, 0.78)","(0.84, 0.87)","(0.47, 0.53)","(0.79, 0.91)","(0.95, 0.95)","(0.96, 0.97)","(0.9, 0.91)","(0.41, 0.52)"
5-5 Hold-out,"(0.97, 0.99)","(0.67, 0.74)","(0.83, 0.87)","(0.47, 0.5)","(0.82, 0.88)","(0.95, 0.95)","(0.96, 0.98)","(0.9, 0.91)","(0.35, 0.46)"
Full,0.989978,0.758741,0.873843,0.505771,0.83871,0.952261,0.969842,0.903086,0.60177
# of labels,5,2,4,3,2,2,2,5,21
# of instance,898,286,1728,1473,155,3163,8124,12960,339


Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.

## Answers to Question 1 and 4
---

- Question 1
  - Based on the code results which I printed out above, IG tends to prefer dataset which has an unevenly distributed IG (large variance). For a single dataset, higher IG trends to affect the final decision (decide which class does the instance belongs to) more. For example, in dataset nursery.csv, attribute 'health' has a extremely high IG which boost the decision making for high accuacy. On the other hand, for evenly large IG, it will give a relatively bad prediction which refer to the CMC.csv.
  - **Strange Results**
    - For dataset hypothyroid.csv, although it has a relatively low IG but the prediction has a pretty high accuracy.
        - Since the IG only consider how good is a single attribute when making decison. However, NB tring to combine several independent attributes to make the final decison. Therefore, the IG is low but acc remains high.
    - For dataset primary-tumor.csv, it has two very high IG but has a low acc.
        - The reason for that is it contain a huge proportion of missing values for those two features. 
- Question 4
  - I've implemented the following evaluation method
    1. Full-test and full-train
    2. Hold-out (by default, 8-2 split, but tested on different splits (9-1, 7-3, 5-5) as well distributions are kept)
    3. random Sampling (by default, 8-2 split, distributions are not kept)
  - When comparing method 1 and 2, for dataset
    - For most of the dataset, the acc for different methods are in the similar range. Those datasets are large enough to learn the pattern within the datasets. As we keep the distribution of data when extracting training data from the dataset for method 2, the performance (acc) are hardly be affected.
    - For samll dataset, the biased data affects 9-1 hold-out more. （as I illustrated the data above, 9-1 hold-out normally has a very large range for small and biased dataset）
        - It is not vary hard to understand, since there are only few testing data, one outlier will result in significatly change in acc.
    - Specific dataset yield some interesting results
      - Breast-cancer and hepatitis
        - Although they got quite similar results in acc. It seems varying in a quite large range for method 2. One of the reason is that small number of instance we have. As the distributions are kept, another reason for this is that the data are biased. The reason has been explained above.
      - Primary-tumor
        - For this dataset, missing values take a huge count in significant larger acc of method 1(refer to the missing value that I printed out for each attribute above). Lacking of instance also becomes a problem here.

